<a href="https://colab.research.google.com/github/ttnghia962/License_Plate_Regconition/blob/main/Copy_of_License_plate_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytesseract
!pip install ultralytics
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.6/595.6 kB 33.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt    # Used for optical character recognition (OCR) to extract text from images
import plotly.express as px # Creating and displaying interactive plots, such as line plots, bar plots
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet # Parsing and manipulating XML data

from glob import glob   # Inding files and directories that match a specified pattern
from skimage import io  # Reading and writing images
from shutil import copy # Copying files and directories
from sklearn.model_selection import train_test_split 

from tensorflow.keras.models import Model   # Creating a deep learning model
from tensorflow.keras.callbacks import TensorBoard  # Logging training information and visualizing it in TensorBoard
from tensorflow.keras.applications import InceptionResNetV2  # Loading the pre-trained InceptionResNetV2 model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input  # Defining different layers in a deep learning model
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Loading and preprocessing images

#Import dataset

In [ ]:
fileName = "/content/data_xml"

In [ ]:
 labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
 for name in os.listdir(fileName):  
    path = glob(f'/content/data_xml/{name}')
    for filename in path:

        info = xet.parse(filename)
        root = info.getroot()
        member_object = root.find('object')
        labels_info = member_object.find('bndbox')
        xmin = int(labels_info.find('xmin').text)
        xmax = int(labels_info.find('xmax').text)
        ymin = int(labels_info.find('ymin').text)
        ymax = int(labels_info.find('ymax').text)

        labels_dict['filepath'].append(filename)
        labels_dict['xmin'].append(xmin)
        labels_dict['xmax'].append(xmax)
        labels_dict['ymin'].append(ymin)
        labels_dict['ymax'].append(ymax)

FileNotFoundError: ignored

In [ ]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)

In [ ]:
df.info()

In [ ]:
filename_xml = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/content/data_img',filename_image)
    return filepath_image
getFilename(filename_xml)

In [ ]:
img_path = list(df['filepath'].apply(getFilename))
img_path[:10]

- Veritfy dataset

In [ ]:
'''img_num_check = int(input())

file_path = img_path[img_num_check] 
img = cv2.imread(file_path) #read the image

img = io.imread(file_path) #Read the image
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10))
fig.add_shape(type='rect',
              x0=df['xmin'][img_num_check], x1=df['xmax'][img_num_check], 
              y0=df['ymin'][img_num_check], y1=df['ymax'][img_num_check], 
              xref='x', yref='y',line_color='cyan')'''

#Data preprocessing

In [ ]:
labels = df.iloc[:,1:].values # Targeting all our values in array selecting all columns

data = []
output = []
for ind in range(len(img_path)):

    img = img_path[ind]
    img_arr = cv2.imread(img)
    h,w,d = img_arr.shape

    # Pre-procesing images
    load_image = load_img(img,target_size=(224,224)) # Resizing image to 244x244
    load_image_arr = img_to_array(load_image) # Converting image into a NumPy array with the color channels
    norm_load_image_arr = load_image_arr/255.0 # Feature scalling to each color channel have a value between 0 and 1

    # Normalization to labels
    xmin, xmax, ymin, ymax = labels[ind]
    nxmin, nxmax = xmin/w, xmax/w
    nymin, nymax = ymin/h, ymax/h
    label_norm = (nxmin, nxmax, nymin, nymax) # Normalized output
    
    data.append(norm_load_image_arr)
    output.append(label_norm)

#Splitting train and test set


In [ ]:
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 0.8, test_size = 0.2,
                                                    random_state = 0)

In [ ]:
df.shape

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

#Object Detection Model

##Initialize the InceptionResNetV2 model with pre-trained weights from ImageNet

In [ ]:
inception_resnet = InceptionResNetV2(
    weights="imagenet", 
    include_top=False, # Exclude the final fully connected layer
    input_tensor=Input(shape=(224,224,3)) # Set the input size as 224x224 pixels with 3 color channels
)


headmodel = inception_resnet.output # Retrieve the output from the InceptionResNetV2 model
headmodel = Flatten()(headmodel) # Add a Flatten layer to convert the output into a 1-dimensional vector
headmodel = Dense(500, activation="relu")(headmodel) # Add a fully connected layer with 500 hidden units and ReLU activation
headmodel = Dense(250, activation="relu")(headmodel) # Add another fully connected layer with 250 hidden units and ReLU activation
headmodel = Dense(4, activation='sigmoid')(headmodel)
# Add a final fully connected layer with 4 hidden units and sigmoid activation
# Sigmoid activation is used to predict the 4 attributes of the region of interest


# Create the complete model by specifying the inputs and outputs
model = Model(
    inputs=inception_resnet.input, 
    outputs=headmodel
) 

In [ ]:
# Compile the model
model.compile(
    loss='mse', # Use mean squared error as the loss function
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4) # Use Adam optimizer with a learning rate of 1e-4
)
model.summary() # Print a summary of the model's architecture

##Training InceptionResNetV2 model

In [ ]:
# Create a TensorBoard callback to visualize training progress and metrics
tfb = TensorBoard('object_detection') 

training_model = model.fit(
    x=X_train, y=y_train,  # Training data
    batch_size=10,  # Number of samples per gradient update
    epochs=140,  # Number of training epochs
    validation_data=(X_test, y_test),  # Validation data
    callbacks=[tfb]  # List of callbacks to be used during training
)

In [ ]:
model.save('./License_plate_recognition.h5')